In [1]:
import geemap 
import os 
import ee

## Draw map centered on Ivory Coast
- We can draw a region of interest to spatially filter the data 

In [2]:
map = geemap.Map(center=(7.5, -5.5), zoom=6)
map

Map(center=[7.5, -5.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…

In [3]:
roi = map.user_roi
roi.getInfo()

{'type': 'Polygon',
 'coordinates': [[[-9.430375, 3.902618],
   [-1.477536, 3.902618],
   [-1.477536, 11.000512],
   [-9.430375, 11.000512],
   [-9.430375, 3.902618]]]}

In [4]:
#*
 # Function to mask clouds using the Sentinel-2 QA band
 # @param {ee.Image} image Sentinel-2 image
 # @return {ee.Image} cloud masked Sentinel-2 image
 #
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)


dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(roi)

# print(dataset.getInfo())

dataset = dataset.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                 .map(maskS2clouds)

In [5]:
visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

map.setCenter(-5.5, 7.5, 7)
map.addLayer(dataset.median(), visualization, 'RGB')
map

Map(bottom=4335.0, center=[7.5, -5.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…

In [6]:
nir = dataset.median().select('B5')
red = dataset.median().select('B4')
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

In [7]:
ndvi_viz = {
    'min': 0.,
    'max': .4,
    'bands':['NDVI'], 
}

In [8]:
map.setCenter(-5.5, 7.5, 7)
map.addLayer(ndvi, ndvi_viz, 'false color composite')
map

Map(bottom=4178.0, center=[7.5, -5.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…

In [9]:
roi = map.user_roi
roi.getInfo()

{'type': 'Polygon',
 'coordinates': [[[-8.318947, 5.235141],
   [-8.099255, 5.235141],
   [-8.099255, 5.388289],
   [-8.318947, 5.388289],
   [-8.318947, 5.235141]]]}

In [10]:
image = dataset.median()
image_clip = image.clip(roi)

filename = 'D:/Books/geopy-notebooks/images/ndvi_median.tif'

geemap.ee_export_image(image_clip, filename=filename, region=roi, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to D:\Books\geopy-notebooks\images\ndvi_median.tif
